In [779]:
import pandas as pd 
import matplotlib as mpl 
import matplotlib.pyplot as plt 
import seaborn as sns 
import scipy.stats as stats
import plotly.express as px
import datetime

mpl.rc('font',family='Malgun Gothic')

In [781]:
df_sales = pd.read_csv('Sales_Data05.csv')
df_member = pd.read_csv('Member_Data.csv',encoding='cp949')
df_product = pd.read_csv('Product_Data.csv')

In [783]:
df_sales.head()

,회원번호,회원상태,구매수량,구매금액,주문일시,배송시작일,배송완료일,사용 적립금,사용 포인트 네이버,주문취소여부,주문시간,제품번호
0,1032097472,정상회원,1.0,7083,2021-01-02,NaN,NaN,0,0,주문취소,오후 12:60,100021783V2_1337
1,1032097472,정상회원,1.0,29865,2021-01-02,2021-01-02,2021-01-02,0,0,NaN,오후 12:60,100022137V2_1606
2,1032097472,정상회원,1.0,23164,2021-01-02,2021-01-02,2021-01-03,0,0,NaN,오후 12:60,100021452V2_1113
3,369152832,정상회원,1.0,16655,2021-01-02,2021-01-02,2021-01-03,0,0,NaN,오후 12:60,10002931V2_708
4,1032097472,정상회원,1.0,8423,2021-01-02,NaN,NaN,0,0,주문취소,오후 12:60,100022085V2_1559


In [785]:
df_member.head()

,Unnamed: 0,회원번호,회원상태,성별,나이,등록카드,결혼,구독여부,주소지,세부주소지
0,0,18764160,정상회원,여,68,농협중앙회,기혼,False,서울특별시,성동구
1,1,18792000,정상회원,남,83,연결앱결제,NaN,False,강원도,강릉시
2,2,18942336,정상회원,여,39,신한은행,기혼,False,인천광역시,중구
3,3,18949760,정상회원,여,73,기업은행,NaN,NaN,강원도,홍천군
4,4,19391488,정상회원,여,52,연결앱결제,기혼,False,대전광역시,중구


In [787]:
df_product.head()

,제품번호,물품명,물품대분류,물품중분류,상품중량
0,100021V2_0,2단무늬컵,식기/편백,자기,1p
1,100022V2_1,7곡딸기롤과자,과자,스낵,100g/10개입
2,100023V2_2,7곡참식,식사대용,선식/생식,700g
3,100024V2_3,가리비,생물수산,패류/갑각류,1.5kg
4,100025V2_3,가리비,생물수산,패류/갑각류,1kg


## 재구매율

재구매율 = (두 번 이상 구매한 고객 수/전체 고객 수) * 100

In [790]:
# '주문일시' 열을 날짜 형식으로 변환
df_sales['주문일시'] = pd.to_datetime(df_sales['주문일시'])

# '회원번호'별 구매 횟수 계산
df_sales['구매횟수'] = 1
customer_order_count = df_sales.pivot_table(
    index=['회원번호','제품번호'],values='구매횟수',aggfunc='sum')

customer_order_count

구매횟수
회원번호       제품번호                 
18764160   100021182V2_892     1
           100021211V2_918     1
           100021235V2_939     1
           10002134V2_106      1
           10002158V2_128      1
...                          ...
1670620864 10002805V2_612      1
           10002819V2_621      1
           10002869V2_665      1
           10002897V2_680      1
           1000296V2_74        2

[373551 rows x 1 columns]

In [792]:
# '구매횟수'가 2 이상인 고객 수 계산
customer_order_count_2 = customer_order_count[customer_order_count['구매횟수'] >= 2].index.get_level_values('회원번호').nunique()

customer_order_count_2

7997

In [794]:
# 전체 고객 수 
customer_total = df_sales['회원번호'].value_counts().shape[0]

# 재구매율 계산
re_order_rate = (customer_order_count_2 / customer_total) * 100
print('재구매율:', round(re_order_rate, 2), '%')



재구매율: 63.77 %


## 고객 이탈률

- 회원상태 : 정상회원/탈퇴
- 고객 이탈률 = (탈퇴 회원 수 / 전체 회원 수) * 100

In [797]:
# 탈퇴 회원 수
member_out = df_member[df_member['회원상태'] == '탈퇴'].shape[0]
print('탈퇴 회원 수:', member_out)

# 전체 회원 수
member_total = df_member['회원상태'].shape[0]
print('전체 회원 수:', member_total)

# 고객 이탈률
out_rate = (member_out/member_total) * 100
print('고객 이탈률: ', round(out_rate,2), '%')

탈퇴 회원 수: 156
전체 회원 수: 12540
고객 이탈률:  1.24 %


## 배송 지연율

- 주문일시와 배송완료일의 차이가 2일 초과할 경우 배송이 지연되었다고 판단
- 배송 지연율 = (지연된 배송 건수 / 전체 배송 건수) * 100

In [800]:
# '배송시작일', '배송완료일' 열을 날짜 형식으로 변환
df_sales['배송시작일'] = pd.to_datetime(df_sales['배송시작일'], errors='coerce')
df_sales['배송완료일'] = pd.to_datetime(df_sales['배송완료일'], errors='coerce')

In [802]:
# 배송지연 여부 (2일 초과면 지연)
df_sales['배송지연'] = (df_sales['배송완료일'] - df_sales['주문일시']).dt.days > 2
df_sales['배송지연'] = df_sales['배송지연'].astype(int)  # 1(지연), 0(정상)

# 확인
df_sales.head()

,회원번호,회원상태,구매수량,구매금액,주문일시,배송시작일,배송완료일,사용 적립금,사용 포인트 네이버,주문취소여부,주문시간,제품번호,구매횟수,배송지연
0,1032097472,정상회원,1.0,7083,2021-01-02,NaT,NaT,0,0,주문취소,오후 12:60,100021783V2_1337,1,0
1,1032097472,정상회원,1.0,29865,2021-01-02,2021-01-02,2021-01-02,0,0,NaN,오후 12:60,100022137V2_1606,1,0
2,1032097472,정상회원,1.0,23164,2021-01-02,2021-01-02,2021-01-03,0,0,NaN,오후 12:60,100021452V2_1113,1,0
3,369152832,정상회원,1.0,16655,2021-01-02,2021-01-02,2021-01-03,0,0,NaN,오후 12:60,10002931V2_708,1,0
4,1032097472,정상회원,1.0,8423,2021-01-02,NaT,NaT,0,0,주문취소,오후 12:60,100022085V2_1559,1,0


In [804]:
# '배송지연' 건수 계산
dealy_order = (df_sales['배송지연'] == 1).sum()

print ('배송지연건수:', dealy_order)

배송지연건수: 31489


In [806]:
# 전체 배송 건수 계산
total_order = df_sales['배송지연'].shape[0]
print ('전체배송건수:', total_order)

# 배송 지연률 계산
dealy_order_rate = (dealy_order / total_order) * 100
print ('배송지연률:',dealy_order_rate,'%')

전체배송건수: 668111
배송지연률: 4.713138984390318 %


## 정기구독률
- 전체 회원 중 구독가입을 한 고객의 비율
- 구독 회원 수 / 전체 회원 수) * 100

In [816]:
# '구독여부' 컬럼 구성 카테고리 확인
df_member['구독여부'].value_counts(dropna=False)

# 전체 회원 수 계산
member_total = df_member.shape[0]
print ('전체 회원 수:', member_total)

# 구독한 회원 수 계산
member_sub = df_member[df_member['구독여부'] == True].shape[0]
print ('정기구독 회원 수:', member_sub)

# 정기구독률 계산
sub_rate = (member_sub / member_total) * 100
print ('정기구독률:', round(sub_rate, 2), '%')

전체 회원 수: 12540
정기구독 회원 수: 1753
정기구독률: 13.98 %


## 유령고객 비율
- 탈퇴하지 않은 회원 중 최근 n개월 간 주문이 없는 고객
- 회원상태분류: 정상회원/탈퇴/탈퇴처리중/탈퇴신청
- '정상회원'에 속하는 고객을 기준으로 함
- (주어진 데이터의 가장 최신 주문일) - (각 회원별로 마지막 주문일)

In [819]:
df_sales

,회원번호,회원상태,구매수량,구매금액,주문일시,배송시작일,배송완료일,사용 적립금,사용 포인트 네이버,주문취소여부,주문시간,제품번호,구매횟수,배송지연
0,1032097472,정상회원,1.0,7083,2021-01-02,NaT,NaT,0,0,주문취소,오후 12:60,100021783V2_1337,1,0
1,1032097472,정상회원,1.0,29865,2021-01-02,2021-01-02,2021-01-02,0,0,NaN,오후 12:60,100022137V2_1606,1,0
2,1032097472,정상회원,1.0,23164,2021-01-02,2021-01-02,2021-01-03,0,0,NaN,오후 12:60,100021452V2_1113,1,0
3,369152832,정상회원,1.0,16655,2021-01-02,2021-01-02,2021-01-03,0,0,NaN,오후 12:60,10002931V2_708,1,0
4,1032097472,정상회원,1.0,8423,2021-01-02,NaT,NaT,0,0,주문취소,오후 12:60,100022085V2_1559,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
668106,1523874368,정상회원,1.0,166554,2021-10-19,2021-10-20,2021-10-20,0,0,NaN,오전 01:00,100022543V2_1912,1,0
668107,402835520,정상회원,1.0,12635,2021-10-23,2021-10-23,2021-10-24,0,0,NaN,오전 01:00,10002432V2_327,1,0
668108,402835520,정상회원,1.0,7849,2021-10-23,2021-10-23,2021-10-23,0,0,NaN,오전 01:00,100022317V2_1725,1,0
668109,1544381312,정상회원,1.0,7275,2021-10-27,2021-10-27,2021-10-28,6270,0,NaN,오전 01:00,10002723V2_563,1,0


In [821]:
df_sales['회원상태'].value_counts()

회원상태
정상회원     660563
탈퇴         7401
탈퇴처리중       131
탈퇴신청         14
Name: count, dtype: int64

In [823]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 668111 entries, 0 to 668110
Data columns (total 14 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   회원번호        668111 non-null  int64         
 1   회원상태        668109 non-null  object        
 2   구매수량        668111 non-null  float64       
 3   구매금액        668111 non-null  int64         
 4   주문일시        668111 non-null  datetime64[ns]
 5   배송시작일       637476 non-null  datetime64[ns]
 6   배송완료일       637476 non-null  datetime64[ns]
 7   사용 적립금      668111 non-null  int64         
 8   사용 포인트 네이버  668111 non-null  int64         
 9   주문취소여부      30635 non-null   object        
 10  주문시간        668111 non-null  object        
 11  제품번호        668111 non-null  object        
 12  구매횟수        668111 non-null  int64         
 13  배송지연        668111 non-null  int32         
dtypes: datetime64[ns](3), float64(1), int32(1), int64(5), object(4)
memory usage: 68.8+ MB


In [825]:
from datetime import timedelta

In [831]:
def latest_date_order(df_sales, month):
    # '주문일시' 컬럼을 날짜 형식으로 변환
    df_sales['주문일시'] = pd.to_datetime(df_sales['주문일시'], errors='coerce')

    # 주어진 데이터의 최신 주문일 조회
    latest_date_data = df_sales['주문일시'].max()

    # 각 회원별 마지막 주문일 조회
    latest_date_member = df_sales.groupby('회원번호')['주문일시'].max().reset_index()

    # '정상회원' 고객 추출
    normal_member = df_sales[df_sales['회원상태'] == '정상회원']

    # n개월 이내 확인
    period = latest_date_data - timedelta(days=month*30)

    # 유령고객 조회
    ghost_member = latest_date_member[latest_date_member['주문일시'] < period]

    # 유령고객 비율 계산
    ghost_member_ratio = (ghost_member.shape[0] / normal_member['회원번호'].nunique()) * 100

    return ghost_member_ratio

month = int(input('최근 주문이 없는 고객 조회(기간을 월 단위로 입력하세요.):'))
ghost_ratio = latest_date_order(df_sales, month)
print('유령고객 비율:', round(ghost_ratio, 2), '%')

최근 주문이 없는 고객 조회(기간을 월 단위로 입력하세요.): 3


유령고객 비율: 30.0 %


## 물품 대분류별 배송 지연율

In [833]:
# df_sales와 df_product를 제품번호 기준으로 병합
df_merged = df_sales.merge(df_product, on='제품번호', how='left')

# 물품대분류별 배송지연률 계산
delay_rate_by_category = df_merged.groupby('물품대분류')['배송지연'].agg(['sum', 'count'])
delay_rate_by_category['배송지연률'] = (delay_rate_by_category['sum'] / delay_rate_by_category['count']) * 100

delay_rate_by_category

,sum,count,배송지연률
물품대분류,,,
가루,543,11805,4.599746
건강일반,253,5707,4.433152
건어물,471,9842,4.785613
견과,193,3221,5.991928
과실주,3,131,2.290076
과일,778,17400,4.471264
과일채소,648,13477,4.808192
과자,1464,31179,4.695468
기름/식초,310,7156,4.332029


## 배송지연율과 물품 대분류의 상관관계 가설검정

In [835]:
import pandas as pd
import scipy.stats as stats

# 물품대분류와 배송지연의 교차표 생성
contingency_table = pd.crosstab(df_merged['물품대분류'], df_merged['배송지연'])

# Chi-Square Test 수행
stats.chi2_contingency(contingency_table)

Chi2ContingencyResult(statistic=54.02132718654482, pvalue=0.6239620283710816, dof=58, expected_freq=array([[1.12486139e+04, 5.56386057e+02],
       [5.43802116e+03, 2.68978842e+02],
       [9.37813286e+03, 4.63867139e+02],
       [3.06918979e+03, 1.51810207e+02],
       [1.24825788e+02, 6.17421207e+00],
       [1.65799138e+04, 8.20086183e+02],
       [1.28418103e+04, 6.35189741e+02],
       [2.97094904e+04, 1.46950960e+03],
       [6.81872777e+03, 3.37272226e+02],
       [9.52868610e-01, 4.71313898e-02],
       [1.95909786e+03, 9.69021375e+01],
       [1.08055300e+03, 5.34469961e+01],
       [1.18146179e+04, 5.84382103e+02],
       [8.17465981e+03, 4.04340193e+02],
       [1.01080302e+04, 4.99969783e+02],
       [4.50306648e+04, 2.22733522e+03],
       [1.46541664e+04, 7.24833644e+02],
       [3.69141300e+03, 1.82587004e+02],
       [8.44622736e+03, 4.17772640e+02],
       [2.19531399e+04, 1.08586009e+03],
       [1.34192486e+04, 6.63751363e+02],
       [1.35888592e+04, 6.72140751e+02]

## 배송지연율과 주소지의 상관관계 가설검정

In [841]:
# df_member에서 '회원번호', '주소지', '세부주소지' 열만 선택
df_member_selected = df_member[['회원번호', '주소지', '세부주소지']]

# df_sales와 df_member_selected를 회원번호를 기준으로 병합
df_merged = pd.merge(df_sales, df_member_selected, on='회원번호', how='inner')

df_merged

,회원번호,회원상태,구매수량,구매금액,주문일시,배송시작일,배송완료일,사용 적립금,사용 포인트 네이버,주문취소여부,주문시간,제품번호,구매횟수,배송지연,주소지,세부주소지
0,1032097472,정상회원,1.0,7083,2021-01-02,NaT,NaT,0,0,주문취소,오후 12:60,100021783V2_1337,1,0,경기도,화성시
1,1032097472,정상회원,1.0,29865,2021-01-02,2021-01-02,2021-01-02,0,0,NaN,오후 12:60,100022137V2_1606,1,0,경기도,화성시
2,1032097472,정상회원,1.0,23164,2021-01-02,2021-01-02,2021-01-03,0,0,NaN,오후 12:60,100021452V2_1113,1,0,경기도,화성시
3,369152832,정상회원,1.0,16655,2021-01-02,2021-01-02,2021-01-03,0,0,NaN,오후 12:60,10002931V2_708,1,0,울산광역시,남구
4,1032097472,정상회원,1.0,8423,2021-01-02,NaT,NaT,0,0,주문취소,오후 12:60,100022085V2_1559,1,0,경기도,화성시
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
668106,1523874368,정상회원,1.0,166554,2021-10-19,2021-10-20,2021-10-20,0,0,NaN,오전 01:00,100022543V2_1912,1,0,경상남도,창원시
668107,402835520,정상회원,1.0,12635,2021-10-23,2021-10-23,2021-10-24,0,0,NaN,오전 01:00,10002432V2_327,1,0,경상남도,김해시
668108,402835520,정상회원,1.0,7849,2021-10-23,2021-10-23,2021-10-23,0,0,NaN,오전 01:00,100022317V2_1725,1,0,경상남도,김해시
668109,1544381312,정상회원,1.0,7275,2021-10-27,2021-10-27,2021-10-28,6270,0,NaN,오전 01:00,10002723V2_563,1,0,충청남도,아산시


### 주소지별 배송지연율 계산

In [860]:
# 주소지별 배송지연율 계산
delivery_delay_rate = df_merged.groupby('주소지')['배송지연'].mean()

delivery_delay_rate

주소지
강원         0.000000
강원도        0.047145
경기         0.052553
경기도        0.046832
경상남도       0.045730
경상북도       0.048000
광주         0.062963
광주광역시      0.047890
대구광역시      0.047653
대전광역시      0.044927
부산광역시      0.048016
서울         0.043478
서울특별시      0.046688
세종특별자치시    0.049554
울산광역시      0.046526
인천광역시      0.048358
전라남도       0.048101
전라북도       0.047494
제주특별자치도    0.043497
충청남도       0.047345
충청북도       0.048312
Name: 배송지연, dtype: float64

In [856]:
contingency_table = pd.crosstab(df_merged['주소지'], df_merged['배송지연'])
stats.chi2_contingency(contingency_table)

Chi2ContingencyResult(statistic=17.37007014309404, pvalue=0.6288291819884981, dof=20, expected_freq=array([[2.28689275e+01, 1.13107252e+00],
       [2.96505174e+04, 1.46648265e+03],
       [6.34612738e+02, 3.13872624e+01],
       [1.13961583e+05, 5.63641712e+03],
       [6.46571281e+04, 3.19787190e+03],
       [3.68837685e+04, 1.82423146e+03],
       [2.57275434e+02, 1.27245658e+01],
       [1.82656030e+04, 9.03397045e+02],
       [3.61329054e+04, 1.78709458e+03],
       [2.49633401e+04, 1.23465991e+03],
       [7.38628243e+04, 3.65317572e+03],
       [2.19160555e+01, 1.08394450e+00],
       [7.59029232e+04, 3.75407681e+03],
       [6.51859720e+03, 3.22402796e+02],
       [2.22009642e+04, 1.09803577e+03],
       [2.49461884e+04, 1.23381160e+03],
       [1.89183203e+04, 9.35679740e+02],
       [2.82688530e+04, 1.39814702e+03],
       [6.55004198e+03, 3.23958020e+02],
       [2.77743124e+04, 1.37368757e+03],
       [2.61134566e+04, 1.29154343e+03]]))